Football Injury Datasets Source

https://github.com/nflverse/nfl_data_py

https://nflreadr.nflverse.com/reference/load_injuries.html

https://nflreadr.nflverse.com/reference/load_rosters.html

https://nflreadr.nflverse.com/reference/load_players.html

https://nflreadr.nflverse.com/reference/load_combine.html

https://nflreadr.nflverse.com/articles/dictionary_injuries.html

https://nflreadr.nflverse.com/articles/dictionary_rosters.html

https://nflreadr.nflverse.com/articles/dictionary_combine.html

In [10]:
# import nfl_data_py as nfl

# combined = nfl.import_combine_data([2009, 2024])
# cleancombined = nfl.clean_nfl_data(combined)
# print("COMBINE DATA")
# print("====================================")
# print(cleancombined.columns.tolist())

# rosters = nfl.import_seasonal_rosters([2009, 2024])
# cleanrosters = nfl.clean_nfl_data(rosters)
# print("ROSTER DATA")
# print("====================================")
# print(cleanrosters.columns.tolist())

# ids = nfl.import_ids()
# cleanids = nfl.clean_nfl_data(ids)
# print("ID DATA")
# print("====================================")
# print(cleanids.columns.tolist())

# injuries = nfl.import_injuries([2009, 2024])
# cleaninjuries = nfl.clean_nfl_data(injuries)
# print("INJURY DATA")
# print("====================================")
# print(cleaninjuries.columns.tolist())

In [ ]:
# import pandas as pd
# import nfl_data_py as nfl

# # ---------------------------
# # 1. Import Data and Standardize Keys
# # ---------------------------
# years = list(range(2009, 2025))

# # Import and clean data
# combine_df = nfl.clean_nfl_data(nfl.import_combine_data(years=years))
# rosters_df = nfl.clean_nfl_data(nfl.import_seasonal_rosters(years=years))
# injuries_df = nfl.clean_nfl_data(nfl.import_injuries(years=years))

# # Create a common key (using player_name for combine/roster and full_name for injuries)
# combine_df['player_key'] = combine_df['player_name'].str.lower().str.strip()
# rosters_df['player_key'] = rosters_df['player_name'].str.lower().str.strip()
# injuries_df['player_key'] = injuries_df['full_name'].str.lower().str.strip()

# # ---------------------------
# # 2. Prepare Injuries Data
# # ---------------------------
# # Rename injuries columns to avoid conflicts
# injuries_df = injuries_df.rename(columns={
#     'season': 'season_injury', 
#     'week': 'week_injury',
#     'position': 'position_injury'
# })

# # Define valid injury types (all lowercase)
# valid_injury_types = [
#     "knee", "ankle", "hamstring", "shoulder", "foot", "concussion",
#     "groin", "back", "calf", "hip", "neck", "toe", "quadricep", "elbow",
#     "hand", "rib", "wrist", "thumb", "abdomen", "head", "finger", "achilles",
#     "shin", "pectoral", "forearm", "heel", "biceps", "fibula"
# ]

# # Create one-hot indicator columns based on practice injury descriptions.
# # (Using both 'practice_primary_injury' and 'practice_secondary_injury')
# for inj in valid_injury_types:
#     col_name = f"injury_{inj}"
#     injuries_df[col_name] = (
#         injuries_df['practice_primary_injury'].str.lower().str.contains(inj, na=False) |
#         injuries_df['practice_secondary_injury'].str.lower().str.contains(inj, na=False)
#     ).astype(int)

# # List of injury indicator column names (these will be our target columns)
# injury_type_cols = [f"injury_{inj}" for inj in valid_injury_types]

# # Filter out rows that don't mention any valid injury type
# injuries_df = injuries_df[injuries_df[injury_type_cols].sum(axis=1) > 0]

# # Sort injuries by season_injury and week_injury (earlier events first)
# injuries_df = injuries_df.sort_values(by=['season_injury', 'week_injury'])

# # Compute cumulative injury counts for each type for each player
# for inj in valid_injury_types:
#     cum_col = f"cum_injury_{inj}"
#     injuries_df[cum_col] = injuries_df.groupby('player_key')[f"injury_{inj}"].cumsum()

# # ---------------------------
# # 3. Merge Datasets
# # ---------------------------
# # Merge roster and injury data on player_key and matching season (roster 'season' with injuries 'season_injury')
# roster_injury_df = pd.merge(
#     rosters_df,
#     injuries_df,
#     left_on=['player_key', 'season'],
#     right_on=['player_key', 'season_injury'],
#     how='outer',
#     suffixes=('_roster', '_injury')
# )

# # Merge in combine data on player_key (drop combine's 'season' to avoid conflict)
# merged_df = pd.merge(
#     roster_injury_df,
#     combine_df.drop(columns=['season']),
#     on='player_key',
#     how='outer',
#     suffixes=('', '_combine')
# )

# # ---------------------------
# # 4. Cleanup and One-Hot Encode Position
# # ---------------------------
# # Drop columns we no longer need
# cols_to_drop = ['report_status', 'practice_primary_injury', 'practice_secondary_injury', 'prev_injury_count', 'position_injury']
# required_columns = ['height', 'weight', 'years_exp', 'age', 'practice_primary_injury']
# merged_df = merged_df.dropna(subset=required_columns)
# merged_df = merged_df.drop(columns=[col for col in cols_to_drop if col in merged_df.columns])

# # One-hot encode the player's position (from roster data; assumed column 'position' exists)
# position_dummies = pd.get_dummies(merged_df['position'], prefix='pos')
# merged_df = pd.concat([merged_df, position_dummies], axis=1)

# # ---------------------------
# # 5. Subset Final Columns with Injury Indicators at the End
# # ---------------------------
# # Define features (all columns except the injury indicator ones)
# # Here we choose: 'height', 'weight', 'years_exp', 'age', cumulative injury counts, and position dummies.
# feature_columns = ['height', 'weight', 'years_exp', 'age'] + \
#                   [f"cum_injury_{inj}" for inj in valid_injury_types] + \
#                   list(position_dummies.columns)
# # Target columns (the injury indicator one-hot columns)
# target_columns = injury_type_cols

# # Set the desired column order: features first, then targets
# desired_columns = feature_columns + target_columns

# final_subset = merged_df[desired_columns]

# # ---------------------------
# # 6. Export and Display
# # ---------------------------
# final_subset.to_csv("train.csv", index=False)
# print("Data exported to train.csv")
# print("Final subset column names:")
# print(final_subset.columns.tolist())

# # ---------------------------
# # Splitting into Features and Targets
# # ---------------------------
# # To split into features (X) and targets (y), you could do:
# X = final_subset[feature_columns]
# y = final_subset[target_columns]

# print("\nFeatures (first few rows):")
# print(X.head())
# print("\nTargets (first few rows):")
# print(y.head())

In [ ]:
# import pandas as pd
# from sklearn.model_selection import train_test_split

# # Load the dataset that was previously exported to train.csv
# data = pd.read_csv("train.csv")

# # Randomly split the data into 80% training and 20% testing sets.
# # Setting random_state ensures reproducibility.
# train_data, test_data = train_test_split(data, test_size=0.20, random_state=42)

# # Export the splits to separate CSV files
# train_data.to_csv("train_data.csv", index=False)
# test_data.to_csv("test_data.csv", index=False)

# print("Training and testing data have been saved as 'train_data.csv' and 'test_data.csv', respectively.")

In [ ]:
# import pandas as pd
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import GridSearchCV
# import joblib

# # Load the pre‐split datasets
# train_data = pd.read_csv("train_data.csv")
# test_data = pd.read_csv("test_data.csv")

# # The first 59 columns are features, the remaining are targets.
# X_train = train_data.iloc[:, :59]
# y_train = train_data.iloc[:, 59:]
# X_test  = test_data.iloc[:, :59]
# y_test  = test_data.iloc[:, 59:]

# # Define a parameter grid for hyperparameter tuning
# param_grid = {
#     'n_estimators': [500, 1000],
#     'max_depth': [None],
#     'min_samples_split': [2],
#     'min_samples_leaf': [1]
# }

# # Create a RandomForestRegressor model
# rf = RandomForestRegressor(random_state=42)

# # Set up GridSearchCV for multioutput regression
# grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring='r2', n_jobs=-1, verbose=2)
# grid_search.fit(X_train, y_train)

# print("Best parameters:", grid_search.best_params_)
# print("Best cross-validation R^2 score:", grid_search.best_score_)

# # Use the best estimator to predict and evaluate on the test set
# best_model = grid_search.best_estimator_
# test_score = best_model.score(X_test, y_test)
# print("Test R^2 score:", test_score)

# # Persist (export) the best model
# joblib.dump(best_model, "RandomForestRegressor3.joblib")
# print("Tuned model saved to 'RandomForestRegressor3.joblib'.")

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time= 5.5min
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time= 5.5min
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time= 5.5min
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=1000; total time= 9.9min
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=1000; total time= 9.9min
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=1000; total time= 9.9min
Best parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 1000}
Best cross-validation R^2 score: 0.5379267872628422
Test R^2 score: 0.579156882855993
Tuned model saved to 'RandomForestRegressor3.joblib'.


In [ ]:
# import numpy as np
# import pandas as pd
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.model_selection import GridSearchCV, train_test_split
# from sklearn.metrics import r2_score
# import joblib

# # Load the pre‐split datasets
# train_data = pd.read_csv("train_data.csv")
# test_data = pd.read_csv("test_data.csv")

# # The first 59 columns are features, the remaining are targets.
# X_train = train_data.iloc[:, :59]
# y_train = train_data.iloc[:, 59:]
# X_test  = test_data.iloc[:, :59]
# y_test  = test_data.iloc[:, 59:]

# # Define a parameter grid for KNeighborsRegressor
# param_grid = {
#     'n_neighbors': [3, 5, 7, 9, 11, 13],
#     'weights': ['uniform', 'distance'],
#     'p': [1, 2]  # p=1: Manhattan distance, p=2: Euclidean distance
# }

# # Create the KNeighborsRegressor instance
# knn = KNeighborsRegressor()

# print("Starting GridSearchCV")
# # Set up GridSearchCV with 5-fold cross-validation and R² as the scoring metric.
# grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='r2', n_jobs=-1)

# # Run the grid search on the training data
# grid_search.fit(X_train, y_train)

# # Output the best parameters and the best cross-validation R² score
# print("Best parameters found:", grid_search.best_params_)
# print("Best cross-validation R² score:", grid_search.best_score_)

# # Retrieve the best estimator and evaluate on the test set
# best_knn = grid_search.best_estimator_
# test_r2 = best_knn.score(X_test, y_test)
# print("Test set R² score:", test_r2)

# # Persist (export) the best model
# joblib.dump(best_knn, "KNearestNeighbours.joblib")
# print("Tuned model saved to 'KNearestNeighbours.joblib'.")

In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.experimental import enable_hist_gradient_boosting  # noqa
# from sklearn.ensemble import HistGradientBoostingRegressor
# from sklearn.multioutput import MultiOutputRegressor
# from sklearn.model_selection import GridSearchCV, train_test_split
# from sklearn.metrics import r2_score
# import joblib

# # Load the pre-split data
# train_data = pd.read_csv("train_data.csv")
# test_data = pd.read_csv("test_data.csv")

# # Assume the first 59 columns are features, remaining are continuous injury risks
# X_train = train_data.iloc[:, :59].values
# y_train = train_data.iloc[:, 59:].values
# X_test  = test_data.iloc[:, :59].values
# y_test  = test_data.iloc[:, 59:].values

# # Initialize the base HistGradientBoostingRegressor
# hgb = HistGradientBoostingRegressor(random_state=42)

# # Wrap it in MultiOutputRegressor to handle multioutput regression
# model = MultiOutputRegressor(hgb)

# # Define a parameter grid; note the prefix for the underlying regressor
# param_grid = {
#     'estimator__max_iter': [100, 200],
#     'estimator__learning_rate': [0.01, 0.1, 0.2],
#     'estimator__max_depth': [None, 5, 10]
# }

# # Set up GridSearchCV
# grid_search = GridSearchCV(estimator=model, 
#                            param_grid=param_grid, 
#                            cv=3, 
#                            scoring='r2', 
#                            n_jobs=2, 
#                            verbose=2)

# # Fit the grid search on the training data
# grid_search.fit(X_train, y_train)

# print("Best parameters:", grid_search.best_params_)
# print("Best cross-validation R^2 score:", grid_search.best_score_)

# # Evaluate the best estimator on the test set
# best_model = grid_search.best_estimator_
# test_score = best_model.score(X_test, y_test)
# print("Test R^2 score:", test_score)

# # Persist (export) the best model using joblib
# joblib.dump(best_model, "HistGradientBoostingRegressor.joblib")
# print("Tuned model saved to 'HistGradientBoostingRegressor.joblib'.")


In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.ensemble import ExtraTreesRegressor
# from sklearn.metrics import r2_score
# import joblib

# # Load the pre-split datasets
# train_data = pd.read_csv("train_data.csv")
# test_data = pd.read_csv("test_data.csv")

# # Assume the first 59 columns are features, and the remaining columns are the continuous injury targets.
# X_train = train_data.iloc[:, :59].values
# y_train = train_data.iloc[:, 59:].values
# X_test  = test_data.iloc[:, :59].values
# y_test  = test_data.iloc[:, 59:].values

# # Initialize the ExtraTreesRegressor with the given parameters:
# etr = ExtraTreesRegressor(n_estimators=50, 
#                            max_depth=None, 
#                            min_samples_split=2, 
#                            min_samples_leaf=1, 
#                            random_state=69)

# print("Starting process")
# # Fit the model on training data
# etr.fit(X_train, y_train)

# # Evaluate on the test set
# y_pred = etr.predict(X_test)
# test_score = r2_score(y_test, y_pred)
# print("Test R^2 score:", test_score)

# # Persist the model using joblib
# joblib.dump(etr, "ExtraTreesRegressor4.joblib")
# print("Model saved to 'ExtraTreesRegressor4.joblib'.")


Starting process
Test R^2 score: 0.6022060455868832
Model saved to 'ExtraTreesRegressor4.joblib'.


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import joblib

test_data = pd.read_csv("test_data.csv")
X_test  = test_data.iloc[:, :59]
y_test  = test_data.iloc[:, 59:]

rf_model = joblib.load('RandomForestRegressor.joblib')
rf2_model = joblib.load('RandomForestRegressor2.joblib')
knn_model = joblib.load('KNearestNeighbours.joblib')
etr_model = joblib.load('ExtraTreesRegressor.joblib')
etr3_model = joblib.load('ExtraTreesRegressor3.joblib')
etr4_model = joblib.load('ExtraTreesRegressor4.joblib')

# List of models to evaluate
models = {
    "RandomForestRegressor": rf_model,
    "RandomForestRegressor2": rf2_model,
    "KNeighborsRegressor": knn_model,
    "ExtraTreesRegressor": etr_model,
    "ExtraTreesRegressor3": etr3_model,
    "ExtraTreesRegressor4": etr4_model
}

# Evaluate each model's R² score on the test set
for name, model in models.items():
    predictions = model.predict(X_test)
    score = r2_score(y_test, predictions)
    print(f"{name} R² score: {score:.4f}")


RandomForestRegressor R² score: 0.5780
RandomForestRegressor2 R² score: 0.5547
KNeighborsRegressor R² score: 0.4344


/Users/heuryn/anaconda3/envs/Hackalytics25/lib/python3.11/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but ExtraTreesRegressor was fitted without feature names
  warnings.warn(


ExtraTreesRegressor R² score: 0.5100


/Users/heuryn/anaconda3/envs/Hackalytics25/lib/python3.11/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but ExtraTreesRegressor was fitted without feature names
  warnings.warn(


ExtraTreesRegressor3 R² score: 0.6044


/Users/heuryn/anaconda3/envs/Hackalytics25/lib/python3.11/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but ExtraTreesRegressor was fitted without feature names
  warnings.warn(


ExtraTreesRegressor4 R² score: 0.6022
